In [23]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [24]:
#get data from Johns Hopkins University Center for Systems Science and Engineering (JHU CCSE)
def get_data(start_date, end_date):
    result = pd.DataFrame()
    temp_date = pd.to_datetime(start_date)
    e_date = pd.to_datetime(end_date)
    while temp_date <= e_date:
        try:
            temp_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{:02d}-{:02d}-{}.csv".format(temp_date.month, temp_date.day, temp_date.year))
            temp_df["date"] = temp_date
            result = result.append(temp_df, ignore_index = True)
        except:
            pass
        temp_date = temp_date + pd.Timedelta(days = 1)
    return result

In [25]:
df = get_data("2020-01-22", "today")
df

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,2020-01-22
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,2020-01-22
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,2020-01-22
...,...,...,...,...,...,...,...
2475,"Seattle, WA",US,2020-02-09T07:03:04,1.0,0.0,1.0,2020-02-26
2476,"Tempe, AZ",US,2020-02-25T21:23:03,1.0,0.0,1.0,2020-02-26
2477,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0,2020-02-26
2478,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0,2020-02-26


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 7 columns):
Province/State    1778 non-null object
Country/Region    2480 non-null object
Last Update       2480 non-null object
Confirmed         2461 non-null float64
Deaths            2039 non-null float64
Recovered         2092 non-null float64
date              2480 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 135.8+ KB


In [27]:
df_co = df[~df["Country/Region"].isin(["Mainland China", "Others"])].groupby(["Country/Region", "date"])[["Confirmed", "Deaths", "Recovered"]].sum().reset_index().sort_values(["Country/Region", "date"])
df_co.head()

,Country/Region,date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-02-24,1.0,0.0,0.0
1,Afghanistan,2020-02-25,1.0,0.0,0.0
2,Afghanistan,2020-02-26,1.0,0.0,0.0
3,Algeria,2020-02-25,1.0,0.0,0.0
4,Algeria,2020-02-26,1.0,0.0,0.0


In [28]:
co_code = pd.read_csv("data/iso-country-codes.csv")
co_code.head()

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS


In [29]:
df_merge = pd.merge(left = df_co, right = co_code, how = "left", left_on = "Country/Region", right_on = "English short name lower case")
df_merge

,Country/Region,date,Confirmed,Deaths,Recovered,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,2020-02-24,1.0,0.0,0.0,Afghanistan,AF,AFG,4.0,ISO 3166-2:AF
1,Afghanistan,2020-02-25,1.0,0.0,0.0,Afghanistan,AF,AFG,4.0,ISO 3166-2:AF
2,Afghanistan,2020-02-26,1.0,0.0,0.0,Afghanistan,AF,AFG,4.0,ISO 3166-2:AF
3,Algeria,2020-02-25,1.0,0.0,0.0,Algeria,DZ,DZA,12.0,ISO 3166-2:DZ
4,Algeria,2020-02-26,1.0,0.0,0.0,Algeria,DZ,DZA,12.0,ISO 3166-2:DZ
...,...,...,...,...,...,...,...,...,...,...
916,Vietnam,2020-02-22,16.0,0.0,14.0,Vietnam,VN,VNM,704.0,ISO 3166-2:VN
917,Vietnam,2020-02-23,16.0,0.0,14.0,Vietnam,VN,VNM,704.0,ISO 3166-2:VN
918,Vietnam,2020-02-24,16.0,0.0,14.0,Vietnam,VN,VNM,704.0,ISO 3166-2:VN
919,Vietnam,2020-02-25,16.0,0.0,16.0,Vietnam,VN,VNM,704.0,ISO 3166-2:VN


In [30]:
unknown_co = list(df_merge[df_merge["Alpha-2 code"].isnull()]["Country/Region"].unique())
print(unknown_co)
# code_list = ["CI", "MO", "KR", "GB", "US"]
# mapper = dict(zip(unknown_co, code_list))
mapper = {'Ivory Coast': 'CI',
    'Macau': 'MO',
    'North Macedonia': 'MK',
    'South Korea': 'KR',
    'UK': 'UK',
    'US': 'US'}

['Ivory Coast', 'Macau', 'North Macedonia', 'South Korea', 'UK', 'US']


In [31]:
# add countrycode manually for unmapped country
df_merge.loc[df_merge["Country/Region"].isin(unknown_co), "Alpha-2 code"] = df_merge.loc[df_merge["Country/Region"].isin(unknown_co), "Country/Region"].map(mapper)

In [32]:
df_merge["Image"] = "https://www.countryflags.io/"+df_merge["Alpha-2 code"]+"/shiny/64.png"
df_merge["date"] = df_merge.date.dt.date
df_merge.to_excel("df_merge.xlsx")

In [33]:
confirmed = df_merge.pivot_table(index = ["Country/Region", "Alpha-2 code", "Image"], columns = "date", values = "Confirmed", aggfunc = "sum", fill_value = 0)
confirmed.to_excel("confirmed.xlsx")